In [ ]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.2.0'
spark_version = 'spark-3.2.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

In [ ]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [ ]:
from pyspark import SparkFiles
# Load in movies_cleaned.csv from S3 into a DataFrame
url = "https://moviepickfiles.s3.us-east-2.amazonaws.com/movies_cleaned3.csv"
spark.sparkContext.addFile(url)

df1 = spark.read.option('header', 'true').csv(SparkFiles.get("movies_cleaned3.csv"), inferSchema=True, sep=',', timestampFormat="mm/dd/yy")
df1.show(10)

In [ ]:
# Count dataframe
df1.count()

In [ ]:
# Display schema
df1.printSchema()

In [ ]:
# Configuration for RDS instance
mode="append"
jdbc_url = "jdbc:postgresql://moviepickdb1.cfpjmecphwzu.us-west-2.rds.amazonaws.com:5432/moviepicktest"
config = {"user":"postgres",
          "password": "postgres",
          "driver":"org.postgresql.Driver"}

In [ ]:
# Write DataFrame to table

df1.write.jdbc(url=jdbc_url, table='movies_cleaned3', mode=mode, properties=config)